In [1]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime
import pandas as pd

In [2]:
url = 'https://legalaffairs.gov.taipei/Content_List.aspx?n=7CBA5467296F9979'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
df_result= pd.DataFrame(columns=['Time', 'Url', 'Question', 'Answer', '點閱數', '資料更新', '資料檢視', '資料維護'])

In [3]:
category_list = []
x = 0

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    categories = soup.find_all('div', class_='list-text content-list')

    for category in categories:
        category_text = category.find('a', title=True)
        if category_text:
            category = category_text['title']
            category_list.append(category)
else:
    print("Failed to fetch the webpage. Status code:", response.status_code)

In [5]:
category_url_list = []
x = 0

for x in categories:
    url = 'https://legalaffairs.gov.taipei/' + x.find('a')['href'] + '&page=1&PageSize=100'
    category_url_list.append(url)

In [7]:
question_list = []
url_list = []

for url in category_url_list:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    questions = soup.find_all('td', class_='CCMS_jGridView_td_Class_1')
    
    for question in questions:
        question_text = question.find('a').text
        question_list.append(question_text)
    for x in questions:
        url = 'https://legalaffairs.gov.taipei/' + x.find('a')['href']
        url_list.append(url)

df_result['Question'] = question_list
df_result['Url'] = url_list

In [9]:
import re
answer_list = []
current_time_list = []
click_list = []
data_update_list = []
data_review_list = []
data_maintainence_list = []

for url in url_list:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    answers = soup.find_all('div', class_='p')
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    clicks = soup.find('span', {'id': 'hitcount'})
    click_value = clicks.text.strip() if clicks else None
    click_numeric = int(re.search(r'\d+', click_value).group()) if click_value else None

    data_update = soup.find_all('li', {'data-index': '2'})
    for li in data_update:
        data_update_value = li.find('span')
        if data_update_value and '資料更新' in data_update_value.text:
            update_info = data_update_value.text.strip().replace('資料更新：', '')
            break

    data_review = soup.find_all('li', {'data-index': '3'})
    for li in data_review:
        data_review_value = li.find('span')
        if data_review_value and '資料檢視' in data_review_value.text:
            review_info = data_review_value.text.strip().replace('資料檢視：', '')
            break

    data_maintainence = soup.find_all('li', {'data-index': '4'})
    for li in data_maintainence:
        data_maintainence_value = li.find('span')
        if data_maintainence_value and '資料維護' in data_maintainence_value.text:
            maintainence_info = data_maintainence_value.text.strip().replace('資料維護：', '')
            break

    for tag in answers:
        answers = tag.text
    
    answer_list.append(answers)
    current_time_list.append(current_time)
    click_list.append(click_numeric)
    data_update_list.append(update_info)
    data_review_list.append(review_info)
    data_maintainence_list.append(maintainence_info)

df_result['Time'] = current_time_list
df_result['Answer'] = answer_list
df_result['點閱數'] = click_list
df_result['資料更新'] = data_update_list
df_result['資料檢視'] = data_review_list
df_result['資料維護'] = data_maintainence_list

In [10]:
df_result.to_csv('法務局QA.csv', index=False)